In [1]:
import pandas as pd
import numpy as np
import string
import xgboost as xgb

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Feature Engineering

In [3]:
def get_title(x):
    name_parts = x.split(',')
    title = name_parts[1].split('.')[0].strip()
    return title

train['Title'] = train['Name'].map(lambda x: get_title(x))

train.loc[train['Title'] == 'Don', 'Title'] = 'Mr'
train.loc[train['Title'] == 'Major', 'Title'] = 'Mr'
train.loc[train['Title'] == 'Capt', 'Title'] = 'Mr'
train.loc[train['Title'] == 'Jonkheer', 'Title'] = 'Mr'
train.loc[train['Title'] == 'Rev', 'Title'] = 'Mr'
train.loc[train['Title'] == 'Col', 'Title'] = 'Mr'
train.loc[train['Title'] == 'Countess', 'Title'] = 'Mrs'
train.loc[train['Title'] == 'Mme', 'Title'] = 'Mrs'
train.loc[train['Title'] == 'Mlle', 'Title'] = 'Miss'
train.loc[train['Title'] == 'Ms', 'Title'] = 'Miss'
train.loc[(train['Title'] == 'Dr') & (train['Sex'] == 'male'), 'Title'] = 'Mr'
train.loc[(train['Title'] == 'Dr') & (train['Sex'] == 'female'), 'Title'] = 'Mrs'
train.loc[train['Title'] == 'the Countess', 'Title'] = 'Mrs'
train.loc[train['Title'] == 'Lady', 'Title'] = 'Miss'
train.loc[train['Title'] == 'Sir', 'Title'] = 'Mr'

test['Title'] = test['Name'].map(lambda x: get_title(x))

test.loc[test['Title'] == 'Don', 'Title'] = 'Mr'
test.loc[test['Title'] == 'Major', 'Title'] = 'Mr'
test.loc[test['Title'] == 'Capt', 'Title'] = 'Mr'
test.loc[test['Title'] == 'Jonkheer', 'Title'] = 'Mr'
test.loc[test['Title'] == 'Rev', 'Title'] = 'Mr'
test.loc[test['Title'] == 'Col', 'Title'] = 'Mr'
test.loc[test['Title'] == 'Countess', 'Title'] = 'Mrs'
test.loc[test['Title'] == 'Mme', 'Title'] = 'Mrs'
test.loc[test['Title'] == 'Mlle', 'Title'] = 'Miss'
test.loc[test['Title']== 'Ms', 'Title'] = 'Miss'
test.loc[(test['Title'] == 'Dr') & (test['Sex'] == 'male'), 'Title'] = 'Mr'
test.loc[(test['Title'] == 'Dr') & (test['Sex'] == 'female'), 'Title'] = 'Mrs'
test.loc[test['Title'] == 'the Countess', 'Title'] = 'Mrs'
test.loc[test['Title'] == 'Lady', 'Title'] = 'Miss'
test.loc[test['Title'] == 'Sir', 'Title'] = 'Mr'

In [4]:
train['Age'] = train['Age'].fillna(train['Age'].median())

train.loc[train.Age <= 1.0, 'age_bins'] = '1s'
train.loc[(train['Age'] > 1.0) & (train['Age'] <= 5.0), 'age_bins'] = '5s'
train.loc[(train['Age'] > 5.0) & (train['Age'] <= 10.0), 'age_bins'] = '10s'
train.loc[(train['Age'] > 10.0) & (train['Age'] <= 15.0), 'age_bins'] = '15s'
train.loc[(train['Age'] > 15.0) & (train['Age'] <= 20.0), 'age_bins'] = '20s'
train.loc[(train['Age'] > 20.0) & (train['Age'] <= 25.0), 'age_bins'] = '25s'
train.loc[(train['Age'] > 25.0) & (train['Age'] <= 30.0), 'age_bins'] = '30s'
train.loc[(train['Age'] > 30.0) & (train['Age'] <= 35.0), 'age_bins'] = '35s'
train.loc[(train['Age'] > 35.0) & (train['Age'] <= 40.0), 'age_bins'] = '40s'
train.loc[(train['Age'] > 40.0) & (train['Age'] <= 45.0), 'age_bins'] = '45s'
train.loc[(train['Age'] > 45.0) & (train['Age'] <= 50.0), 'age_bins'] = '50s'
train.loc[(train['Age'] > 50.0) & (train['Age'] <= 55.0), 'age_bins'] = '55s'
train.loc[(train['Age'] > 55.0), 'age_bins'] = '60s'

test['Age'] = test['Age'].fillna(test['Age'].median())

test.loc[test.Age <= 1.0, 'age_bins'] = '1s'
test.loc[(test['Age'] > 1.0) & (test['Age'] <= 5.0), 'age_bins'] = '5s'
test.loc[(test['Age'] > 5.0) & (test['Age'] <= 10.0), 'age_bins'] = '10s'
test.loc[(test['Age'] > 10.0) & (test['Age'] <= 15.0), 'age_bins'] = '15s'
test.loc[(test['Age'] > 15.0) & (test['Age'] <= 20.0), 'age_bins'] = '20s'
test.loc[(test['Age'] > 20.0) & (test['Age'] <= 25.0), 'age_bins'] = '25s'
test.loc[(test['Age'] > 25.0) & (test['Age'] <= 30.0), 'age_bins'] = '30s'
test.loc[(test['Age'] > 30.0) & (test['Age'] <= 35.0), 'age_bins'] = '35s'
test.loc[(test['Age'] > 35.0) & (test['Age'] <= 40.0), 'age_bins'] = '40s'
test.loc[(test['Age'] > 40.0) & (test['Age'] <= 45.0), 'age_bins'] = '45s'
test.loc[(test['Age'] > 45.0) & (test['Age'] <= 50.0), 'age_bins'] = '50s'
test.loc[(test['Age'] > 50.0) & (test['Age'] <= 55.0), 'age_bins'] = '55s'
test.loc[(test['Age'] > 55.0), 'age_bins'] = '60s'

In [5]:
train['relative_size'] = train['SibSp'] + train['Parch']
train.loc[train['relative_size'] > 0, 'travelled_alone'] = 0 #No
train.loc[train['relative_size'] == 0, 'travelled_alone'] = 1 #Yes

test['relative_size'] = test['SibSp'] + test['Parch']
test.loc[test['relative_size'] > 0, 'travelled_alone'] = 0 #No
test.loc[test['relative_size'] == 0, 'travelled_alone'] = 1 #Yes

In [6]:
train['Cabin'].loc[~train['Cabin'].isnull()] = 1  
train['Cabin'].loc[train['Cabin'].isnull()] = 0 

test['Cabin'].loc[~test['Cabin'].isnull()] = 1  
test['Cabin'].loc[test['Cabin'].isnull()] = 0 

B:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
train['Embarked'] = train['Embarked'].fillna('S')

test['Embarked'] = test['Embarked'].fillna('S')

In [8]:
train_Q1 = train['Fare'].quantile(0.25)
train_Q3 = train['Fare'].quantile(0.75)
train_IQR = train_Q3 - train_Q1

test_Q1 = test['Fare'].quantile(0.25)
test_Q3 = test['Fare'].quantile(0.75)
test_IQR = test_Q3 - test_Q1

train_lower_lim = train_Q1 - 1.5 * train_IQR
train_upper_lim = train_Q3 + 1.5 * train_IQR

test_lower_lim = test_Q1 - 1.5 * test_IQR
test_upper_lim = test_Q3 + 1.5 * test_IQR

train.loc[train['Fare'] > train_upper_lim, 'Fare'] = train_upper_lim
test.loc[test['Fare'] > test_upper_lim, 'Fare'] = test_upper_lim
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [9]:
train = train.drop(['PassengerId', 'Ticket', 'Name', 'Age'], axis=1)

train['travelled_alone'] = train['travelled_alone'].astype(int)

test = test.drop(['PassengerId', 'Ticket', 'Name', 'Age'], axis=1)

test['travelled_alone'] = test['travelled_alone'].astype(int)

In [10]:
from sklearn.preprocessing import OneHotEncoder

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

ohe1 = OneHotEncoder(sparse=False, drop='first')
data_obj1 = ohe1.fit_transform(train[train.columns[train.dtypes=='object']])

obj_df1 = pd.DataFrame(data_obj1,columns=ohe1.get_feature_names(train.columns[train.dtypes=='object']))

train = pd.concat([train[train.columns[train.dtypes!='object']], obj_df1], axis=1)

train = (train-train.min())/(train.max()-train.min())

ohe2 = OneHotEncoder(sparse=False, drop='first')
data_obj2 = ohe2.fit_transform(test[test.columns[test.dtypes=='object']])

obj_df2 = pd.DataFrame(data_obj2,columns=ohe2.get_feature_names(test.columns[test.dtypes=='object']))

test = pd.concat([test[test.columns[test.dtypes!='object']], obj_df2], axis=1)

test = (test-test.min())/(test.max()-test.min())

In [11]:
sex = [col for col in train.columns if 'Sex' in col]
cabin = [col for col in train.columns if 'Cabin' in col]
age_bins = [col for col in train.columns if 'age_bins' in col]
title = [col for col in train.columns if 'Title' in col]

features = ['Pclass', 'Fare', 'relative_size'] + sex + cabin + age_bins + title

# Model

In [12]:
X_train = train[features]
X_test = test[features]
y_train = train['Survived']

In [20]:
from xgboost import XGBClassifier
xgbc = xgb.XGBClassifier(random_state = 1, learning_rate = 0.2)
xgbc.fit(X_train, y_train)
preds = xgbc.predict(X_test)

[20:28:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


B:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [19]:
test_data = pd.read_csv('submission.csv')
submission = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': preds})
submission = submission.astype(int)

submission.to_csv('submission.csv', index = False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
